# Caitlyn Messages

In [12]:
f = open('Caitlyn - Messages.txt', 'r', encoding = 'utf-8')
text = f.read()
print('corpus length:', len(text))
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
text = text.replace("\n\n", " ")
text = text.replace("\n", " ")
text = text.replace("------------------------------------------", " ")
text = text.replace("Conversation with Caitlyn", " ")
text = text.replace("Message received from Caitlyn", " ")
text = text.replace("(SMS)", " ")
text = text.replace("(MMS)", " ")
text = text.replace("Message sent", " ")
text = text.replace("AM", " ")
text = text.replace("PM", " ")
text = text.replace("Messages", " ")
import re
text = re.sub(r'http\S+', '', text)
text = re.sub("[^A-Za-z.']+", ' ', text)
text = text[3:]
text[200:300]

corpus length: 457516
total chars: 113


"gaged. I'm hoping that'll change soon. Thanks for checking in and I'll definitely get back to you on"

In [2]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 100
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 96038


In [3]:
import numpy as np
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer=optimizer)

print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               123904    
_________________________________________________________________
dense (Dense)                (None, 113)               14577     
_________________________________________________________________
activation (Activation)      (None, 113)               0         
Total params: 138,481
Trainable params: 138,481
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
epochs = 10
batch_size = 128

model_structure = model.to_json()
with open("shakes_lstm_model.json", "w") as json_file:
    json_file.write(model_structure)
# for i in range(10):
#     print('*'*10)
#     print('Range ' + str(i + 1) + '/' + str(10))
#     print('*'*10)
#     model.fit(X, y,
#               batch_size = batch_size,
#               epochs = epochs)
#     model.save_weights("shakes_lstm_weights_{}.h5".format(i+1))
#     print('Model saved.')
model.load_weights('shakes_lstm_weights_10.h5')

In [8]:
import random
def sample(preds, temperature = 1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
np.seterr(divide = 'ignore') 

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [10]:
import sys

start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.4, 0.6, 0.8]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(1000):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose = 0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "tired but that's the scaffolding of the first half or so. Serendipity. Did you just write all of tha"
tired but that's the scaffolding of the first half or so. Serendipity. Did you just write all of that I still want to the world and I'm going to be able to the world in the most right. I'm not my sister or almost it's a good thing. I've been a good thing. I'm still a produment is the contine that was that I was pretty says are the stuff I was so working and then I was a bit a couple of your life. I don't know what I can and then there was a good dimpasion that was that I was so say we can do that was a good digh' and then I was so say we can be a good shift it would be a good day. I think I said I really had a good buning and then I get that I was a bit a couple of your sister and the probly is to be a sillen and probably want to the rest of the same way to say that I was so say that I was so working on the sort of me and then there wa